# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 6 2022 2:27PM,245767,10,MSP213691,"Methapharm, Inc.",Released
1,Sep 6 2022 2:24PM,245756,10,0085963684,ISDIN Corporation,Released
2,Sep 6 2022 2:24PM,245756,10,0085963727,ISDIN Corporation,Released
3,Sep 6 2022 2:24PM,245756,10,0085963726,ISDIN Corporation,Released
4,Sep 6 2022 2:24PM,245756,10,0085963739,ISDIN Corporation,Released
5,Sep 6 2022 2:24PM,245756,10,0085963740,ISDIN Corporation,Released
6,Sep 6 2022 2:24PM,245756,10,0085963741,ISDIN Corporation,Released
7,Sep 6 2022 2:24PM,245756,10,0085963743,ISDIN Corporation,Released
8,Sep 6 2022 2:24PM,245756,10,0085963742,ISDIN Corporation,Released
9,Sep 6 2022 2:24PM,245756,10,0085963744,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,245762,Released,1
35,245764,Released,1
36,245765,Released,2
37,245766,Released,5
38,245767,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
245762,NaN,NaN,1.0
245764,NaN,NaN,1.0
245765,NaN,NaN,2.0
245766,NaN,NaN,5.0
245767,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
245647,0.0,0.0,1.0
245654,0.0,1.0,0.0
245659,0.0,0.0,1.0
245689,0.0,5.0,2.0
245704,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
245647,0.0,0,1
245654,0.0,1,0
245659,0.0,0,1
245689,0.0,5,2
245704,0.0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,245647,0.0,0,1
1,245654,0.0,1,0
2,245659,0.0,0,1
3,245689,0.0,5,2
4,245704,0.0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,245647,0.0,,1
1,245654,0.0,1,
2,245659,0.0,,1
3,245689,0.0,5,2
4,245704,0.0,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 6 2022 2:27PM,245767,10,"Methapharm, Inc."
1,Sep 6 2022 2:24PM,245756,10,ISDIN Corporation
70,Sep 6 2022 2:20PM,245757,10,ISDIN Corporation
129,Sep 6 2022 2:19PM,245766,10,"Methapharm, Inc."
134,Sep 6 2022 2:18PM,245755,10,ISDIN Corporation
191,Sep 6 2022 2:18PM,245765,10,Eywa Pharma Inc.
193,Sep 6 2022 2:09PM,245764,10,"Senseonics, Incorporated"
194,Sep 6 2022 1:45PM,245762,20,"Adare Pharmaceuticals, Inc."
195,Sep 6 2022 1:31PM,245759,12,Hush Hush
201,Sep 6 2022 1:27PM,245758,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,Sep 6 2022 2:27PM,245767,10,"Methapharm, Inc.",0.0,,1
1,Sep 6 2022 2:24PM,245756,10,ISDIN Corporation,0.0,,69
2,Sep 6 2022 2:20PM,245757,10,ISDIN Corporation,0.0,,59
3,Sep 6 2022 2:19PM,245766,10,"Methapharm, Inc.",0.0,,5
4,Sep 6 2022 2:18PM,245755,10,ISDIN Corporation,0.0,,57
5,Sep 6 2022 2:18PM,245765,10,Eywa Pharma Inc.,0.0,,2
6,Sep 6 2022 2:09PM,245764,10,"Senseonics, Incorporated",0.0,,1
7,Sep 6 2022 1:45PM,245762,20,"Adare Pharmaceuticals, Inc.",0.0,,1
8,Sep 6 2022 1:31PM,245759,12,Hush Hush,0.0,,6
9,Sep 6 2022 1:27PM,245758,10,"Methapharm, Inc.",1.0,,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 2:27PM,245767,10,"Methapharm, Inc.",1,
1,Sep 6 2022 2:24PM,245756,10,ISDIN Corporation,69,
2,Sep 6 2022 2:20PM,245757,10,ISDIN Corporation,59,
3,Sep 6 2022 2:19PM,245766,10,"Methapharm, Inc.",5,
4,Sep 6 2022 2:18PM,245755,10,ISDIN Corporation,57,
5,Sep 6 2022 2:18PM,245765,10,Eywa Pharma Inc.,2,
6,Sep 6 2022 2:09PM,245764,10,"Senseonics, Incorporated",1,
7,Sep 6 2022 1:45PM,245762,20,"Adare Pharmaceuticals, Inc.",1,
8,Sep 6 2022 1:31PM,245759,12,Hush Hush,6,
9,Sep 6 2022 1:27PM,245758,10,"Methapharm, Inc.",,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 2:27PM,245767,10,"Methapharm, Inc.",1,
1,Sep 6 2022 2:24PM,245756,10,ISDIN Corporation,69,
2,Sep 6 2022 2:20PM,245757,10,ISDIN Corporation,59,
3,Sep 6 2022 2:19PM,245766,10,"Methapharm, Inc.",5,
4,Sep 6 2022 2:18PM,245755,10,ISDIN Corporation,57,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 2:27PM,245767,10,"Methapharm, Inc.",1.0,NaN
1,Sep 6 2022 2:24PM,245756,10,ISDIN Corporation,69.0,NaN
2,Sep 6 2022 2:20PM,245757,10,ISDIN Corporation,59.0,NaN
3,Sep 6 2022 2:19PM,245766,10,"Methapharm, Inc.",5.0,NaN
4,Sep 6 2022 2:18PM,245755,10,ISDIN Corporation,57.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 2:27PM,245767,10,"Methapharm, Inc.",1.0,0.0
1,Sep 6 2022 2:24PM,245756,10,ISDIN Corporation,69.0,0.0
2,Sep 6 2022 2:20PM,245757,10,ISDIN Corporation,59.0,0.0
3,Sep 6 2022 2:19PM,245766,10,"Methapharm, Inc.",5.0,0.0
4,Sep 6 2022 2:18PM,245755,10,ISDIN Corporation,57.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4423504,431.0,10.0
12,491463,6.0,1.0
15,245724,14.0,7.0
16,491463,4.0,0.0
19,491480,9.0,20.0
20,737157,6.0,6.0
21,1228404,4.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4423504,431.0,10.0
1,12,491463,6.0,1.0
2,15,245724,14.0,7.0
3,16,491463,4.0,0.0
4,19,491480,9.0,20.0
5,20,737157,6.0,6.0
6,21,1228404,4.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,431.0,10.0
1,12,6.0,1.0
2,15,14.0,7.0
3,16,4.0,0.0
4,19,9.0,20.0
5,20,6.0,6.0
6,21,4.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,431.0
1,12,Released,6.0
2,15,Released,14.0
3,16,Released,4.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,10.0,1.0,7.0,0.0,20.0,6.0,1.0
Released,431.0,6.0,14.0,4.0,9.0,6.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,10.0,1.0,7.0,0.0,20.0,6.0,1.0
1,Released,431.0,6.0,14.0,4.0,9.0,6.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,10.0,1.0,7.0,0.0,20.0,6.0,1.0
1,Released,431.0,6.0,14.0,4.0,9.0,6.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()